In [1]:
import psycopg2
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import time
import pprint
import metis
import networkit as nk
from networkit import vizbridges
import pickle
import powerlaw

/var/folders/lk/t243gzpx379bzd9mvx8dcsmm0000gp/T/ipykernel_96966/1584475792.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# 파일 불러오기
with open('../../data/df.pkl', 'rb') as f:
    df = pickle.load(f)


In [3]:
# 파일 불러오기
with open('../../data/df_qna.pkl', 'rb') as f:
    df_qna = pickle.load(f)

In [4]:
df['q_creationdate'].min()

Timestamp('2023-08-03 01:04:49.693000')

In [5]:
df_gephi = df[['q_owneruserid', 'a_owneruserid']]

In [6]:
df_gephi.columns = ['Source', 'Target']

In [7]:
node = np.unique(np.append(df_gephi['Source'].values, df_gephi['Target'].values))

In [8]:
# 질문자->답변자 로 edgelist 생성
edgelist = list(zip(df_gephi['Source'], df_gephi['Target']))

In [9]:
di_github = nx.DiGraph()
di_github.add_nodes_from(node)
di_github.add_edges_from(edgelist)

github = nx.from_pandas_edgelist(df,  source='q_owneruserid', target='a_owneruserid')

In [10]:
di_github_nk = nk.nxadapter.nx2nk(di_github)
github_nk = nk.nxadapter.nx2nk(github)

In [11]:
print(di_github_nk.numberOfNodes(), di_github_nk.numberOfEdges())

11392 11996


### Core Decomposition

In [12]:
rm_github_nk = github_nk.removeSelfLoops()

In [13]:
type(rm_github_nk)

NoneType

In [14]:
coreDec = nk.centrality.CoreDecomposition(github_nk)
coreDec.run()
# direct 라 안되는데
# untimeError: Core Decomposition implementation does not support graphs with self-loops. Call Graph.removeSelfLoops() first.

In [15]:
# nk.gephi.pyclient.GephiClient(url='http://116.45.16.200:5999/workspace0', autoflush=False)
gsc = nk.gephi.streaming.GephiStreamingClient(url='http://116.45.16.200:5999/workspace1')
# gsc = nk.gephi.streaming.GephiStreamingClient(url='http://143.248.106.43:7777/workspace1')
# gsc = nk.gephi.streaming.GephiStreamingClient(url='http://143.248.106.43:7777/workspace1')


In [16]:
# test = nk.gephi.streaming.GephiStreamingClient(url='http://localhost:8080/workspace1')

In [17]:
# test = nk.gephi.streaming.GephiStreamingClient(url='http://116.45.16.200:8080/workspace2')

In [18]:
# test.exportGraph(di_github_nk)

In [19]:
gsc.exportGraph(di_github_nk)

Could not connect to the gephi streaming plugin.
Did you start the streaming master server in gephi and provide the name of your workspace?
If the workspace is named 'Workspace 0', the corresponding url is http://localhost:8080/workspace0 (adapt port)


In [20]:
gsc.clearGraph()

Could not connect to the gephi streaming plugin.
Did you start the streaming master server in gephi and provide the name of your workspace?
If the workspace is named 'Workspace 0', the corresponding url is http://localhost:8080/workspace0 (adapt port)


In [21]:
# nk.vizbridges.widgetFromGraph(github_nk, dimension = nk.vizbridges.Dimension.Two, nodeScores = coreDec.scores())

In [22]:
di_github_nk
# 1762

In [23]:
list(di_github.nodes())[126]

112968

### 고군분투한 포인트  
시각화 해서 노드 간의 전반적인 관계를 보고 싶은데, 현재 환경에서 바로 네트워크 그래프를 볼 수 있는 방법이 없음...  
* 서브 그래프를 그리는 방법
    * centrality기반으로 주요노드들 추출한 다음에 해당 노드를 포함하는 그래프를 서브 그래프로 그려보기(closeness/betweenness)
    * community detection 결과로 나뉘어진 그룹기준으로 서브 그래프 그려보기(방향 없는 그래프로 변환 후 그려야 함)

